In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, classification_report
from sklearn.metrics import accuracy_score, classification_report

from aeon.classification.deep_learning import InceptionTimeClassifier

SEED = 42
RNG = np.random.default_rng(SEED)

In [3]:
from src.data_loader import load_sensor_data, load_target

sensor_data = load_sensor_data()   # dict: {sensor_name: DataFrame (2205, timesteps)}
y = load_target()                  # pd.Series (2205,)
y = y.to_numpy()                   # aeon / sklearn lieber ndarray

print("n_sensors:", len(sensor_data))
print("y shape:", y.shape, "classes:", np.unique(y))

n_sensors: 17
y shape: (2205,) classes: [ 73  80  90 100]


In [4]:
def check_nan_inf_sensor_dict(sensor_dict):
    for name, df in sensor_dict.items():
        x = df.to_numpy()
        n_nan = np.isnan(x).sum()
        n_inf = np.isinf(x).sum()
        if n_nan > 0 or n_inf > 0:
            print(f"[WARN] {name}: NaN={n_nan}, Inf={n_inf}")
    print("NaN/Inf check done.")

check_nan_inf_sensor_dict(sensor_data)

NaN/Inf check done.


In [5]:
# =========================================================
# 3) Resampling: all sensors -> fixed length (建议 60)
#    用 np.interp 做“行级别重采样”，不依赖 scipy
# =========================================================
def resample_rowwise_to_len(df: pd.DataFrame, target_len: int = 60) -> np.ndarray:
    """
    df: (n_samples, old_len)
    return: (n_samples, target_len) as float32
    """
    x = df.to_numpy(dtype=np.float32)
    n, old_len = x.shape

    if old_len == target_len:
        return x

    old_idx = np.linspace(0, 1, old_len, dtype=np.float32)
    new_idx = np.linspace(0, 1, target_len, dtype=np.float32)

    out = np.empty((n, target_len), dtype=np.float32)
    for i in range(n):
        out[i] = np.interp(new_idx, old_idx, x[i])
    return out

In [6]:
# =========================================================
# 4) Build multivariate X for aeon: (n_samples, n_channels, n_timepoints)
# =========================================================
TARGET_LEN = 60

# 建议固定一个顺序，确保每次运行通道顺序一致（可复现 + 易解释）
SENSOR_ORDER = list(sensor_data.keys())  # 或你手动写：["PS1","PS2",...,"SE"]

X_channels = []
for s in SENSOR_ORDER:
    arr = resample_rowwise_to_len(sensor_data[s], target_len=TARGET_LEN)  # (2205, 60)
    X_channels.append(arr)

# stack -> (channels, samples, time) then transpose
X = np.stack(X_channels, axis=0)                 # (n_channels, n_samples, 60)
X = np.transpose(X, (1, 0, 2)).astype(np.float32) # (n_samples, n_channels, 60)

print("X shape:", X.shape)  # (2205, 17, 60)

X shape: (2205, 17, 60)


In [7]:
# =========================================================
# 5) Normalization (z-normalization per sample per channel over time axis)
#    对齐你以前作业的 "Signal Normalisierung"
# =========================================================
def z_norm_per_sample_channel(X, eps=1e-8):
    # X: (n, c, t)
    mean = X.mean(axis=2, keepdims=True)
    std = X.std(axis=2, keepdims=True)
    return (X - mean) / (std + eps)

X = z_norm_per_sample_channel(X)

In [17]:
def plot_loss_and_accuracy(history, model_name="Model"):
    """
    Visualisiert den Trainingsverlauf (Loss & Accuracy) eines aeon DL-Modells.
    Hinweis: aeon speichert nur Trainingsmetriken (kein Validierungsset).
    """

    fig, ax = plt.subplot_mosaic(
        [["loss", "accuracy"]],
        figsize=(12, 4)
    )

    # Loss
    ax["loss"].plot(
        history.epoch,
        history.history["loss"],
        label="train loss"
    )
    ax["loss"].set_xlabel("Epoch")
    ax["loss"].set_ylabel("Loss")
    ax["loss"].set_title(f"{model_name} – Loss history")
    ax["loss"].legend()

    # Accuracy
    ax["accuracy"].plot(
        history.epoch,
        history.history["accuracy"],
        label="train accuracy"
    )
    ax["accuracy"].set_xlabel("Epoch")
    ax["accuracy"].set_ylabel("Accuracy")
    ax["accuracy"].set_title(f"{model_name} – Accuracy history")
    ax["accuracy"].legend()

    plt.tight_layout()
    plt.show()


In [11]:
def evaluate_aeon_model(model, X, y, dataset_name="Val"):
    # 1) Vorhersage Wahrscheinlichkeiten
    proba = model.predict_proba(X)

    print(f"\n=== {dataset_name} Evaluation ===")
    print("proba shape:", proba.shape, "dtype:", proba.dtype)
    print("NaN in proba:", np.isnan(proba).any())
    print("Inf in proba:", np.isinf(proba).any())

    # 2) NaN/Inf/0 Zeilen finden, insbondere für Softmax-Ausgaben bei CNNs
    bad_rows = np.where(
        (~np.isfinite(proba).all(axis=1)) | (proba.sum(axis=1) == 0)
    )[0]

    print("bad proba rows:", len(bad_rows))

    # 3) Bad rows behandeln: Gleichverteilung annehmen
    if len(bad_rows) > 0:
        print("bad row indices (first 20):", bad_rows[:20])
        i = bad_rows[0]
        print(f"{dataset_name} sample min/max/mean/std:",
              X[i].min(), X[i].max(), X[i].mean(), X[i].std())

        proba[bad_rows] = 1.0 / proba.shape[1]

    # 4) Vorhersagen
    y_pred = np.argmax(proba, axis=1)

    # 5) Evaluation
    acc = accuracy_score(y, y_pred)
    print(f"{dataset_name} Accuracy: {acc:.4f}")
    print(classification_report(y, y_pred, digits=3, zero_division=0))

    return acc

In [12]:
# =========================================================
# 6) Training: 3 splits (按老师要求)
#    - 每次 split: train/test = 80/20
#    - 模型内部再用 validation_split=0.2 做 early stopping / 监控
# =========================================================

from aeon.classification.deep_learning import InceptionTimeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score

def run_inceptiontime_3splits_aeon_style(X, y, base_seed=42):
    results = []

    for k in range(3):
        rs = base_seed + k
        print("\n" + "="*60)
        print(f"Split {k+1}/3 | random_state={rs}")
        print("="*60)

        # -------------------------------------------------
        # 1) 切分 Train / Test
        # -------------------------------------------------
        X_train, X_test, y_train, y_test = train_test_split(
            X, y,
            test_size=0.2,
            random_state=rs,
            stratify=y
        )

        # -------------------------------------------------
        # 2) 从 Train 再切 Validation
        #    （因为 aeon 训练阶段不支持 validation_split）
        # -------------------------------------------------
        X_train2, X_val, y_train2, y_val = train_test_split(
            X_train, y_train,
            test_size=0.2,
            random_state=rs,
            stratify=y_train
        )

        # -------------------------------------------------
        # 3) 定义 InceptionTime 模型（与你旧作业一致）
        # -------------------------------------------------
        model = InceptionTimeClassifier(
            batch_size=16,
            n_epochs=30,
            n_classifiers=2, # baseline
            verbose=1,
            random_state=rs
        )

        # -------------------------------------------------
        # 4) 训练（只用训练集）
        # -------------------------------------------------
        model.fit(X_train2, y_train2)

        # -------------------------------------------------
        # 5) 训练曲线（loss / accuracy）
        # -------------------------------------------------
        plot_loss_and_accuracy(
            model.history,
            model_name=f"InceptionTime – Split {k+1}"
        )

        # -------------------------------------------------
        # 6) Validation & Test 评估（你自己的函数）
        # -------------------------------------------------
        val_acc = evaluate_aeon_model(
            model, X_val, y_val, dataset_name="Validation"
        )

        test_acc = evaluate_aeon_model(
            model, X_test, y_test, dataset_name="Test"
        )

        # Balanced Accuracy（作业更推荐）
        y_test_pred = model.predict(X_test)
        bal_acc = balanced_accuracy_score(y_test, y_test_pred)

        results.append({
            "split": k + 1,
            "random_state": rs,
            "val_accuracy": val_acc,
            "test_accuracy": test_acc,
            "test_balanced_accuracy": bal_acc
        })

    results_df = pd.DataFrame(results)

    print("\n=== Summary (3 splits) ===")
    print(results_df)

    print("\nTest Balanced Acc mean:",
          results_df["test_balanced_accuracy"].mean().round(4))
    print("Test Balanced Acc std :",
          results_df["test_balanced_accuracy"].std(ddof=1).round(4))

    return results_df


In [18]:
run_inceptiontime_3splits_aeon_style(X, y, base_seed=SEED)


Split 1/3 | random_state=42


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_5       │ (None, 60, 17)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_160 (Conv1D) │ (None, 60, 32)    │        544 │ input_layer_5[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_30    │ (None, 60, 17)    │          0 │ input_layer_5[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_161 (Conv1D) │ (None, 60, 32)    │     40,960 │ conv1d_160[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_162 (Conv1D) │ (None, 60, 32)    │     20,480 │ conv1d_160[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_163 (Conv1D) │ (None, 60, 32)    │     10,240 │ conv1d_160[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_164 (Conv1D) │ (None, 60, 32)    │        544 │ max_pooling1d_30… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_30      │ (None, 60, 128)   │          0 │ conv1d_161[0][0], │
│ (Concatenate)       │                   │            │ conv1d_162[0][0], │
│                     │                   │            │ conv1d_163[0][0], │
│                     │                   │            │ conv1d_164[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 128)   │        512 │ concatenate_30[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_40       │ (None, 60, 128)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_165 (Conv1D) │ (None, 60, 32)    │      4,096 │ activation_40[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling1d_31    │ (None, 60, 128)   │          0 │ activation_40[0]… │
│ (MaxPooling1D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_166 (Conv1D) │ (None, 60, 32)    │     40,960 │ conv1d_165[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_167 (Conv1D) │ (None, 60, 32)    │     20,480 │ conv1d_165[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_168 (Conv1D) │ (None, 60, 32)    │     10,240 │ conv1d_165[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_169 (Conv1D) │ (None, 60, 32)    │      4,096 │ max_pooling1d_31… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_31      │ (None, 60, 128)   │          0 │ conv1d_166[0][0], │
│ (Concatenate)       │                   │            │ conv1d_167[0][0], │
│                     │                   │            │ conv1d_168[0][0], │
│                     │                   │            │ conv1d_169[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 60, 128)   │        512 │ concatenate_31[0… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_41       │ (None, 60, 128)   │          0 │ batch_normalizat

 Total params: 495,300 (1.89 MB)

 Trainable params: 493,252 (1.88 MB)

...

 Total params: 495,300 (1.89 MB)

 Trainable params: 493,252 (1.88 MB)

 Non-trainable params: 2,048 (8.00 KB)

Epoch 1/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 10s 57ms/step - accuracy: 0.6088 - loss: 0.9864 - learning_rate: 0.0010
Epoch 2/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 4s 48ms/step - accuracy: 0.7505 - loss: 0.5906 - learning_rate: 0.0010
Epoch 3/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 6s 72ms/step - accuracy: 0.8023 - loss: 0.4796 - learning_rate: 0.0010
Epoch 4/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.8441 - loss: 0.3779 - learning_rate: 0.0010
Epoch 5/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - accuracy: 0.9001 - loss: 0.2717 - learning_rate: 0.0010
Epoch 6/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 54ms/step - accuracy: 0.9150 - loss: 0.2175 - learning_rate: 0.0010
Epoch 7/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.9490 - loss: 0.1437 - learning_rate: 0.0010
Epoch 8/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.9646 - loss: 0.1038 - learning_rate: 0.0010
Epoch 9/30
89/89 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.9674 - loss: 0.0912 - learning_rate: 0.0010
Epoch 10/30
89/89 

AttributeError: 'InceptionTimeClassifier' object has no attribute 'history'